In [17]:
import numpy as np
import pandas as pd

In [18]:
df = pd.read_csv(r"C:\Users\DEBABRATA\Desktop\Datasets\My_play.csv")

In [19]:
df.head()

,Outlook,Temperature,Humidity,Windy,Play
0,Sunny,Hot,High,f,No
1,Sunny,Hot,High,t,No
2,Overcast,Hot,High,f,Yes
3,Rainy,Mild,High,f,Yes
4,Rainy,Cool,Normal,f,Yes


In [20]:
x=df.drop([df.columns[-1]], axis = 1)

y= df[df.columns[-1]]

In [21]:
features = list(x.columns)
print(features)

['Outlook', 'Temperature', 'Humidity', 'Windy']


In [22]:
x_train=x
y_train=y
train_size=x.shape[0]
num_feats = x.shape[1]


In [23]:
likelihoods={}
class_priors={}
pred_priors={}


In [24]:
for feature in features:
    likelihoods[feature]= {}
    pred_priors[feature] = {}
    
    for feat_val in np.unique(x_train[feature]):
        pred_priors[feature].update({feat_val:0})
        
        for outcome in np.unique(y_train):
            likelihoods[feature].update({feat_val+'_'+outcome:0})
            class_priors.update({outcome:0})

In [25]:
print(class_priors)

{'No': 0, 'Yes': 0}


In [26]:
print(pred_priors)

{'Outlook': {'Overcast': 0, 'Rainy': 0, 'Sunny': 0}, 'Temperature': {'Cool': 0, 'Hot': 0, 'Mild': 0}, 'Humidity': {'High': 0, 'Normal': 0}, 'Windy': {'f': 0, 't': 0}}


In [27]:
print(likelihoods)

{'Outlook': {'Overcast_No': 0, 'Overcast_Yes': 0, 'Rainy_No': 0, 'Rainy_Yes': 0, 'Sunny_No': 0, 'Sunny_Yes': 0}, 'Temperature': {'Cool_No': 0, 'Cool_Yes': 0, 'Hot_No': 0, 'Hot_Yes': 0, 'Mild_No': 0, 'Mild_Yes': 0}, 'Humidity': {'High_No': 0, 'High_Yes': 0, 'Normal_No': 0, 'Normal_Yes': 0}, 'Windy': {'f_No': 0, 'f_Yes': 0, 't_No': 0, 't_Yes': 0}}


In [28]:
for outcome in np.unique(y_train):
    outcome_count = sum(y_train == outcome)
    class_priors[outcome] = outcome_count/train_size

In [29]:
print(class_priors)

{'No': 0.35714285714285715, 'Yes': 0.6428571428571429}


In [32]:

for feature in features:
    for outcome in np.unique(y_train):
        outcome_count= sum(y_train==outcome)
        feat_likelihood = x_train[feature][y_train[y_train==outcome].index.values.tolist()].value_counts().to_dict()
        print(feat_likelihood)
        for feat_val, count in feat_likelihood.items():
            likelihoods[feature][feat_val + '_' + outcome]= count/outcome_count

{'Sunny': 3, 'Rainy': 2}
{'Overcast': 4, 'Rainy': 3, 'Sunny': 2}
{'Mild': 2, 'Hot': 2, 'Cool': 1}
{'Mild': 4, 'Cool': 3, 'Hot': 2}
{'High': 4, 'Normal': 1}
{'Normal': 6, 'High': 3}
{'t': 3, 'f': 2}
{'f': 6, 't': 3}


In [33]:
print(likelihoods)

{'Outlook': {'Overcast_No': 0, 'Overcast_Yes': 0.4444444444444444, 'Rainy_No': 0.4, 'Rainy_Yes': 0.3333333333333333, 'Sunny_No': 0.6, 'Sunny_Yes': 0.2222222222222222}, 'Temperature': {'Cool_No': 0.2, 'Cool_Yes': 0.3333333333333333, 'Hot_No': 0.4, 'Hot_Yes': 0.2222222222222222, 'Mild_No': 0.4, 'Mild_Yes': 0.4444444444444444}, 'Humidity': {'High_No': 0.8, 'High_Yes': 0.3333333333333333, 'Normal_No': 0.2, 'Normal_Yes': 0.6666666666666666}, 'Windy': {'f_No': 0.4, 'f_Yes': 0.6666666666666666, 't_No': 0.6, 't_Yes': 0.3333333333333333}}


In [34]:

for feature in features:
    feat_vals= x_train[feature].value_counts().to_dict()
    print(feat_vals)
    for feat_val, count in feat_vals.items():
        pred_priors[feature][feat_val]= count/train_size

{'Sunny': 5, 'Rainy': 5, 'Overcast': 4}
{'Mild': 6, 'Cool': 4, 'Hot': 4}
{'High': 7, 'Normal': 7}
{'f': 8, 't': 6}


In [36]:
print(pred_priors)

{'Outlook': {'Overcast': 0.2857142857142857, 'Rainy': 0.35714285714285715, 'Sunny': 0.35714285714285715}, 'Temperature': {'Cool': 0.2857142857142857, 'Hot': 0.2857142857142857, 'Mild': 0.42857142857142855}, 'Humidity': {'High': 0.5, 'Normal': 0.5}, 'Windy': {'f': 0.5714285714285714, 't': 0.42857142857142855}}


In [40]:
# Given User Input and Predict Correct Output

results = []
qu = np.array([['Sunny','Mild','Normal','t']])
qu= np.array(qu)

m=0
p=0

for query in qu:
    probs_outcome={}
    for outcome in np.unique(y_train):
        prior = class_priors[outcome]
        likelihood =1
        evidence =1
        
        
        for feat, feat_val in zip(features, query):
            likelihood*=likelihoods[feat][feat_val + '_'+ outcome]
            evidence *= pred_priors[feat][feat_val]
            
        posterior = (likelihood * prior+ m*p) / (evidence+m)
        probs_outcome[outcome]= posterior
    print(probs_outcome)
    result= max(probs_outcome, key = lambda X: probs_outcome[X])
    results.append(result)
    
print(results)

{'No': 0.3136, 'Yes': 0.43017832647462273}
['Yes']
